In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

import cv2
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
import random

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# GPU selection (memory)
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
#os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
def data_process(d_dir, split_rate):
    
    num_class = len(d_dir.keys())

    out = {}
    

    for i in range(num_class):
        x_train, x_test = train_test_split(d_dir[i], test_size=split_rate)
        
        if i == 0:
            train_temp = x_train
            test_temp = x_test
        else:
            train_temp.extend(x_train)
            test_temp.extend(x_test)
    
    random.shuffle(train_temp)
    out['train'] = train_temp
    random.shuffle(test_temp)
    out['test'] = test_temp
    
    return out

In [4]:
# Data info.

width = 640
height = 480
channel = 3


# resize
r_w = 512
r_h = 512

total_pix = r_w * r_h * channel

In [5]:
image_dir1 = '/dataset/'
class_name1 = ['untorn', 'torn']

data_dir1 = {}
for i in range(len(class_name1)):
    temp_dir = image_dir1 + class_name1[i]
    img_files = [[os.path.join(temp_dir, f), np.int16(i)] for f in os.listdir(temp_dir) if f.endswith('.png')]
    data_dir1[i] = img_files
print(len(data_dir1[0]), len(data_dir1[1]))

1780 1485


In [6]:
tr_data_dir = data_process(data_dir1, 0.2)

In [7]:
tr_data = np.zeros((len(tr_data_dir['train']), total_pix), dtype=np.float32)
tr_label = np.zeros((len(tr_data_dir['train']), 1), dtype=np.int32)

for i in range(len(tr_data_dir['train'])):
    img = cv2.imread(tr_data_dir['train'][i][0])
    img2 = cv2.resize(img, (r_w, r_h), interpolation=cv2.INTER_CUBIC)
    tr_data[i,:] = img2.flatten()
    tr_label[i] = tr_data_dir['train'][i][1]

In [8]:
ev_data = np.zeros((len(tr_data_dir['test']), total_pix), dtype=np.float32)
ev_label = np.zeros((len(tr_data_dir['test']), 1), dtype=np.int32)

for i in range(len(tr_data_dir['test'])):
    img = cv2.imread(tr_data_dir['test'][i][0])
    img2 = cv2.resize(img, (r_w, r_h), interpolation=cv2.INTER_CUBIC)
    ev_data[i,:] = img2.flatten()
    ev_label[i] = tr_data_dir['test'][i][1]

In [9]:
def cnn_model(x, n_class, dropout, reuse, is_training, width=512, height=512):
    with tf.variable_scope('ConvNet', reuse=reuse):
        # RGB data, 3-channel
        input_layer = tf.reshape(x, shape=[-1, height, width, 3])
        
        k_init = tf.contrib.layers.xavier_initializer_conv2d()
        
        conv1 = tf.layers.conv2d(
            inputs=input_layer,
            filters=48,
            kernel_size=[5, 5],
            padding="same",
            activation=tf.nn.relu,
            kernel_initializer=k_init,
            name='conv1')
        pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2, name='pool1')
        # height/2, width/2
        
        conv2 = tf.layers.conv2d(
            inputs=pool1,
            filters=96,
            kernel_size=[5, 5],
            padding="same",
            activation=tf.nn.relu,
            kernel_initializer=k_init,
            name='conv2')
        pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2, name='pool2')
        # height/4, width/4
        
        conv3 = tf.layers.conv2d(
            inputs=pool2,
            filters=96,
            kernel_size=[5, 5],
            padding="same",
            activation=tf.nn.relu,
            kernel_initializer=k_init,
            name='conv3')
        pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2, name='pool3')
        # height/8, width/8
        
        conv4 = tf.layers.conv2d(
            inputs=pool3,
            filters=96,
            kernel_size=[5, 5],
            padding="same",
            activation=tf.nn.relu,
            kernel_initializer=k_init,
            name='conv4')
        pool4 = tf.layers.max_pooling2d(inputs=conv4, pool_size=[2, 2], strides=2, name='pool4')
        # height/16, width/16
        
        conv5 = tf.layers.conv2d(
            inputs=pool4,
            filters=96,
            kernel_size=[5, 5],
            padding="same",
            activation=tf.nn.relu,
            kernel_initializer=k_init,
            name='conv5')
        pool5 = tf.layers.max_pooling2d(inputs=conv5, pool_size=[2, 2], strides=2, name='pool5')
        # height/32, width/32
        
        #pool5_flat = tf.reshape(pool5, [-1, int(height/32) * int(width/32) * 96])
        pool5_flat = tf.contrib.layers.flatten(pool5)
        
        dense1 = tf.layers.dense(pool5_flat, 1024, activation=tf.nn.relu, name='dense1')
        
        dropout = tf.layers.dropout(dense1, rate=dropout, training=is_training, name='drop')
        
        # Output
        out = tf.layers.dense(dropout, n_class, name='output')
        out = tf.nn.softmax(out) if not is_training else out
        
        return out

In [10]:
dropout = 0.5
learning_rate = 0.0001
batch_size = 16
disp_step = 20
n_classes = 2
epochs = 30

tr_steps = int(tr_data.shape[0]/batch_size*epochs)
inner_steps = int((tr_data.shape[0])/batch_size)

In [11]:
with tf.device('/cpu:0'):
    reuse_vars = False
    
    # tf graph input
    X = tf.placeholder(tf.float32, [None, total_pix])
    Y = tf.placeholder(tf.int32, [None, 1])
    
    with tf.device('/gpu:0'):
        # cnn_model(x, n_class, dropout, re_use, is_training, width, height)
        logits_train = cnn_model(X, n_classes, dropout, reuse=reuse_vars,
                                 is_training=True)
        
        logits_test = cnn_model(X, n_classes, dropout, reuse=True,
                                is_training=False)
        
        loss = tf.losses.sparse_softmax_cross_entropy(logits=logits_train, labels=Y)
        #optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        grads = optimizer.compute_gradients(loss)
        
        pred = tf.argmax(input=logits_test, axis=1)
        correct_pred = tf.equal(pred, tf.cast(Y, tf.int64))
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        #accuracy = tf.metrics.accuracy(labels=Y, predictions=pred)
        
        
        prob = tf.nn.softmax(logits_test, name='softmax_tensor')
        
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        
        
        reuse_vars = True
        
    
    
    
    
    
    init_g = tf.global_variables_initializer()
    init_l = tf.local_variables_initializer()

In [12]:
    with tf.Session() as sess:

        sess.run(init_g)
        sess.run(init_l)

        t_count = 1

        for j in range(epochs):

            for i in range(inner_steps):

                tr_batch = {}

                if i < inner_steps-1:
                    tr_batch['data'] = tr_data[i*batch_size:(i+1)*batch_size][:]
                    tr_batch['label'] = tr_label[i*batch_size:(i+1)*batch_size][:]

                else:
                    tr_batch['data'] = tr_data[i*batch_size:][:]
                    tr_batch['label'] = tr_label[i*batch_size:][:]

                sess.run(train_op, feed_dict={X: tr_batch['data'],
                                              Y: tr_batch['label']})

                if t_count % disp_step == 1:

                    loss_tr = sess.run(loss, feed_dict = {X: tr_batch['data'],
                                                          Y: tr_batch['label']})

                    acc_tr = sess.run(accuracy, feed_dict = {X: tr_batch['data'],
                                                             Y: tr_batch['label']})
                    
                    prob_tr = sess.run(prob, feed_dict = {X: tr_batch['data'],
                                                         Y: tr_batch['label']})

                    #print('step %d, training accuracy %f, loss %f' % (t_count, accuracy, loss))
                    print('Training step: ', t_count)
                    print('Accuracy', acc_tr)
                    print('Probability')
                    print(prob_tr)
                    print('Loss: ', loss_tr)
                    #print(acc_tr, loss_tr)

                t_count = t_count+1
                
        print("Training is finished!!!")
        
        
        
        ev_accuracy = np.zeros((len(ev_label), 1))
        for k in range(len(ev_label)):
            ev_batch = {}
            ev_batch['data'] = ev_data[k:(k+1)][:]
            ev_batch['label'] = ev_label[k:(k+1)]
            ev_accuracy[k] = sess.run(accuracy, feed_dict={X: ev_batch['data'],
                                                           Y: ev_batch['label']})

        print(np.mean(ev_accuracy))

Training step:  1
Accuracy 0.5625
Probability
[[0.7310586  0.26894143]
 [0.7310586  0.26894143]
 [0.7310586  0.26894143]
 [0.7310586  0.26894143]
 [0.7310586  0.26894143]
 [0.7310586  0.26894143]
 [0.7310586  0.26894143]
 [0.7310586  0.26894143]
 [0.7310586  0.26894143]
 [0.7310586  0.26894143]
 [0.7310586  0.26894143]
 [0.7310586  0.26894143]
 [0.7310586  0.26894143]
 [0.7310586  0.26894143]
 [0.7310586  0.26894143]
 [0.7310586  0.26894143]]
Loss:  102.28726
Training step:  21
Accuracy 0.625
Probability
[[0.7306523  0.2693477 ]
 [0.6864495  0.3135505 ]
 [0.72959363 0.27040634]
 [0.73077744 0.26922256]
 [0.65385574 0.34614423]
 [0.6810937  0.31890628]
 [0.7283548  0.27164516]
 [0.65091515 0.3490848 ]
 [0.5808306  0.41916943]
 [0.7265085  0.2734915 ]
 [0.7249049  0.2750951 ]
 [0.6718412  0.32815883]
 [0.7305039  0.26949605]
 [0.72306633 0.2769336 ]
 [0.7308448  0.2691552 ]
 [0.73091966 0.26908034]]
Loss:  1.4066256
Training step:  41
Accuracy 0.515625
Probability
[[0.338152   0.66184795

Training step:  361
Accuracy 0.546875
Probability
[[0.73023623 0.2697637 ]
 [0.72287077 0.27712923]
 [0.4618344  0.5381656 ]
 [0.50221306 0.49778694]
 [0.70496804 0.295032  ]
 [0.67798275 0.32201725]
 [0.4785918  0.5214082 ]
 [0.28886086 0.71113914]
 [0.6912157  0.3087843 ]
 [0.6581715  0.34182853]
 [0.6696717  0.33032826]
 [0.30246395 0.6975361 ]
 [0.3665168  0.6334832 ]
 [0.5124572  0.48754278]
 [0.43545187 0.56454813]
 [0.66799897 0.33200106]]
Loss:  0.30269966
Training step:  381
Accuracy 0.4765625
Probability
[[0.7309452  0.26905483]
 [0.4825582  0.5174418 ]
 [0.7238777  0.27612224]
 [0.7024616  0.2975384 ]
 [0.32284158 0.67715836]
 [0.45833454 0.54166543]
 [0.5556281  0.4443719 ]
 [0.72322756 0.27677247]
 [0.6815092  0.3184908 ]
 [0.72937655 0.27062348]
 [0.4094004  0.5905996 ]
 [0.38972035 0.6102797 ]
 [0.32810315 0.6718969 ]
 [0.2964715  0.7035285 ]
 [0.36786115 0.63213885]
 [0.44110087 0.55889916]]
Loss:  0.3455333
Training step:  401
Accuracy 0.5078125
Probability
[[0.3897851

Training step:  721
Accuracy 0.5078125
Probability
[[0.27311718 0.72688276]
 [0.7227194  0.2772807 ]
 [0.72828007 0.27171993]
 [0.7189429  0.2810571 ]
 [0.72135895 0.27864102]
 [0.2747347  0.72526526]
 [0.725166   0.27483398]
 [0.41906443 0.5809356 ]
 [0.28582656 0.7141735 ]
 [0.37540126 0.62459874]
 [0.3524549  0.64754516]
 [0.7294851  0.2705149 ]
 [0.70425355 0.29574642]
 [0.7131463  0.28685367]
 [0.7200783  0.27992174]
 [0.26955256 0.7304475 ]]
Loss:  0.09121127
Training step:  741
Accuracy 0.5078125
Probability
[[0.68863404 0.31136596]
 [0.56358546 0.4364145 ]
 [0.7026541  0.29734585]
 [0.27555564 0.7244444 ]
 [0.31259802 0.687402  ]
 [0.39583516 0.60416484]
 [0.27398002 0.72602004]
 [0.73104376 0.26895624]
 [0.729493   0.270507  ]
 [0.33880305 0.66119695]
 [0.4585347  0.5414653 ]
 [0.7307887  0.2692113 ]
 [0.7127887  0.28721133]
 [0.44037065 0.5596294 ]
 [0.7310585  0.26894152]
 [0.70537704 0.29462296]]
Loss:  0.22016975
Training step:  761
Accuracy 0.5078125
Probability
[[0.40809

Training step:  1081
Accuracy 0.5
Probability
[[0.27297845 0.7270216 ]
 [0.27416012 0.72583985]
 [0.7281985  0.27180144]
 [0.2689417  0.7310583 ]
 [0.7303227  0.26967728]
 [0.7302207  0.2697793 ]
 [0.7310584  0.26894155]
 [0.7310586  0.26894143]
 [0.7303431  0.26965687]
 [0.2689416  0.7310584 ]
 [0.6930702  0.3069299 ]
 [0.34860986 0.65139014]
 [0.7310586  0.26894143]
 [0.4535809  0.54641914]
 [0.28627834 0.7137217 ]
 [0.26894143 0.7310586 ]]
Loss:  0.114792615
Training step:  1101
Accuracy 0.5078125
Probability
[[0.26928395 0.730716  ]
 [0.26894143 0.7310586 ]
 [0.73105854 0.2689415 ]
 [0.7310586  0.26894143]
 [0.27160668 0.72839326]
 [0.26894152 0.7310585 ]
 [0.73105824 0.2689418 ]
 [0.27069008 0.7293099 ]
 [0.7310586  0.26894143]
 [0.73054874 0.2694512 ]
 [0.2689849  0.7310151 ]
 [0.7310585  0.2689415 ]
 [0.3006852  0.69931483]
 [0.28833878 0.7116612 ]
 [0.27473894 0.7252611 ]
 [0.7309814  0.26901862]]
Loss:  0.010798474
Training step:  1121
Accuracy 0.515625
Probability
[[0.7307292

Training step:  1441
Accuracy 0.53125
Probability
[[0.7310586  0.26894143]
 [0.73105854 0.26894143]
 [0.31970292 0.6802971 ]
 [0.33506233 0.6649377 ]
 [0.33755752 0.66244245]
 [0.33759284 0.66240716]
 [0.7310586  0.26894143]
 [0.27069935 0.7293006 ]
 [0.73105824 0.26894173]
 [0.7310586  0.26894143]
 [0.7310586  0.26894143]
 [0.73105836 0.26894164]
 [0.70576483 0.2942351 ]
 [0.2706353  0.7293647 ]
 [0.7310586  0.26894143]
 [0.73100567 0.26899436]]
Loss:  0.03124158
Training step:  1461
Accuracy 0.6953125
Probability
[[0.26904356 0.73095644]
 [0.26971292 0.73028713]
 [0.27010074 0.7298992 ]
 [0.26894432 0.7310557 ]
 [0.2730232  0.7269768 ]
 [0.26895398 0.73104596]
 [0.7310586  0.26894143]
 [0.72974294 0.27025706]
 [0.2689426  0.7310574 ]
 [0.26894167 0.73105836]
 [0.26895833 0.73104167]
 [0.26894143 0.73105854]
 [0.2703686  0.7296314 ]
 [0.2693856  0.7306144 ]
 [0.27472487 0.7252751 ]
 [0.7310586  0.26894143]]
Loss:  0.0033202963
Training step:  1481
Accuracy 0.5703125
Probability
[[0.71

...

Training step:  3241
Accuracy 0.53125
Probability
[[0.7295762  0.27042377]
 [0.26894602 0.73105395]
 [0.26934984 0.7306501 ]
 [0.271997   0.72800297]
 [0.7310361  0.26896387]
 [0.7310586  0.26894143]
 [0.72943413 0.27056584]
 [0.7310585  0.2689415 ]
 [0.26894784 0.73105216]
 [0.2689862  0.73101383]
 [0.731043   0.26895702]
 [0.73104906 0.26895094]
 [0.7310446  0.26895544]
 [0.7310075  0.26899245]
 [0.26894143 0.7310586 ]
 [0.7310585  0.26894152]]
Loss:  0.002757926
Training step:  3261
Accuracy 0.5078125
Probability
[[0.7226635  0.27733645]
 [0.7310586  0.26894143]
 [0.7309813  0.26901868]
 [0.73105335 0.26894665]
 [0.7310586  0.26894143]
 [0.2689428  0.7310572 ]
 [0.26896816 0.73103184]
 [0.26894143 0.7310586 ]
 [0.26894227 0.7310577 ]
 [0.7282691  0.27173087]
 [0.7310389  0.26896107]
 [0.7305639  0.26943606]
 [0.26894772 0.7310522 ]
 [0.2689487  0.7310513 ]
 [0.7310586  0.26894143]
 [0.26894143 0.7310586 ]]
Loss:  0.0043317494
Training step:  3281
Accuracy 0.53125
Probability
[[0.731

Training step:  3601
Accuracy 0.53125
Probability
[[0.73104775 0.2689522 ]
 [0.73091763 0.26908237]
 [0.27040005 0.7296    ]
 [0.2692737  0.7307263 ]
 [0.27323896 0.72676104]
 [0.2715661  0.72843385]
 [0.73105824 0.26894173]
 [0.73105854 0.26894143]
 [0.7310586  0.26894143]
 [0.7310586  0.26894143]
 [0.2694336  0.73056644]
 [0.7310586  0.26894143]
 [0.7310586  0.26894143]
 [0.26894143 0.73105854]
 [0.73103845 0.26896152]
 [0.7310094  0.2689906 ]]
Loss:  0.0008368452
Training step:  3621
Accuracy 0.5703125
Probability
[[0.7310586  0.26894143]
 [0.7310586  0.26894143]
 [0.26894143 0.7310586 ]
 [0.7310586  0.26894143]
 [0.7310586  0.26894143]
 [0.7310586  0.26894143]
 [0.7310586  0.26894143]
 [0.27465054 0.7253494 ]
 [0.7310586  0.26894143]
 [0.7310586  0.26894143]
 [0.7310586  0.26894143]
 [0.26929045 0.73070955]
 [0.268956   0.73104393]
 [0.7310482  0.2689518 ]
 [0.26906687 0.7309331 ]
 [0.7310586  0.26894143]]
Loss:  0.00065402
Training step:  3641
Accuracy 0.5703125
Probability
[[0.73

Training step:  3961
Accuracy 0.5
Probability
[[0.73098737 0.26901257]
 [0.73101985 0.2689801 ]
 [0.7310586  0.26894143]
 [0.7310586  0.26894143]
 [0.26894456 0.73105544]
 [0.7310566  0.2689434 ]
 [0.2692653  0.7307347 ]
 [0.7310584  0.26894155]
 [0.26894143 0.7310586 ]
 [0.7310572  0.26894283]
 [0.26894143 0.7310586 ]
 [0.26898515 0.7310149 ]
 [0.27351207 0.72648793]
 [0.2693861  0.73061395]
 [0.7310586  0.26894143]
 [0.26894143 0.7310586 ]]
Loss:  0.00068357214
Training step:  3981
Accuracy 0.5078125
Probability
[[0.26894143 0.7310586 ]
 [0.7310585  0.26894152]
 [0.7310586  0.26894143]
 [0.7310586  0.26894143]
 [0.73105854 0.26894143]
 [0.26901817 0.73098177]
 [0.7310586  0.26894143]
 [0.26934972 0.7306503 ]
 [0.26894143 0.7310586 ]
 [0.2689415  0.7310585 ]
 [0.26894367 0.73105633]
 [0.7310586  0.26894143]
 [0.7310586  0.26894143]
 [0.72880965 0.27119035]
 [0.73105854 0.26894143]
 [0.26898295 0.73101705]]
Loss:  0.0005371417
Training step:  4001
Accuracy 0.5078125
Probability
[[0.731

Training step:  4321
Accuracy 0.5703125
Probability
[[0.72995377 0.27004626]
 [0.26908657 0.73091346]
 [0.27593935 0.7240607 ]
 [0.7310586  0.26894143]
 [0.73098874 0.2690113 ]
 [0.2760805  0.72391945]
 [0.7310446  0.26895544]
 [0.26898178 0.7310182 ]
 [0.36258256 0.6374174 ]
 [0.27388668 0.72611326]
 [0.2689447  0.7310553 ]
 [0.26894236 0.73105764]
 [0.26943347 0.7305665 ]
 [0.7308169  0.26918307]
 [0.2690864  0.73091364]
 [0.26894146 0.73105854]]
Loss:  0.033192843
Training step:  4341
Accuracy 0.5078125
Probability
[[0.2689729  0.73102707]
 [0.26895925 0.7310408 ]
 [0.7310586  0.26894143]
 [0.2689415  0.73105854]
 [0.73069304 0.269307  ]
 [0.7310586  0.26894143]
 [0.7310586  0.26894143]
 [0.7310586  0.26894143]
 [0.73104566 0.26895434]
 [0.26894143 0.7310586 ]
 [0.73105747 0.26894253]
 [0.26894155 0.7310585 ]
 [0.7310586  0.26894143]
 [0.7199601  0.28003994]
 [0.2689522  0.73104775]
 [0.26894143 0.7310586 ]]
Loss:  0.0020214585
Training step:  4361
Accuracy 0.5078125
Probability
[[0

Training step:  4681
Accuracy 0.5078125
Probability
[[0.2690242  0.7309758 ]
 [0.33114046 0.66885954]
 [0.27209306 0.72790694]
 [0.26894143 0.7310586 ]
 [0.73105365 0.26894638]
 [0.72830874 0.27169132]
 [0.7193667  0.28063327]
 [0.730548   0.26945195]
 [0.26894143 0.7310586 ]
 [0.26898855 0.73101145]
 [0.7310586  0.26894143]
 [0.26894212 0.7310578 ]
 [0.7310586  0.26894143]
 [0.26933092 0.730669  ]
 [0.2698523  0.73014766]
 [0.72596115 0.27403885]]
Loss:  0.022989549
Training step:  4701
Accuracy 0.515625
Probability
[[0.69290715 0.30709282]
 [0.73105854 0.26894143]
 [0.26895612 0.7310438 ]
 [0.27108854 0.72891146]
 [0.268944   0.731056  ]
 [0.26894203 0.73105794]
 [0.7310586  0.26894143]
 [0.26894507 0.7310549 ]
 [0.7310586  0.26894143]
 [0.73105854 0.26894146]
 [0.7310586  0.26894143]
 [0.73086184 0.26913813]
 [0.29913354 0.7008665 ]
 [0.2698523  0.7301477 ]
 [0.7310586  0.26894143]
 [0.7295874  0.27041262]]
Loss:  0.089026295
Training step:  4721
Accuracy 0.6953125
Probability
[[0.2